In [3]:
# from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig
# from prompting.tasks.mock import MockTask, MockRewardConfig
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-31 11:32:32,311	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "required_hash_fields" in "StreamPromptingSynapse" shadows an attribute in parent "StreamingSynapse"
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config[

## LLM Pipeline

In [4]:
# Initialise the LLM we use on the validator
from prompting.llms.vllm_llm import vLLMPipeline
pipeline = vLLMPipeline(llm_model_id="casperhansen/llama-3-8b-instruct-awq", llm_max_allowed_memory_in_gb=20, device="CUDA", quantization=False)

INFO:bittensor: - Available free memory: 44.65 GB - 
INFO:bittensor: - Total gpu memory 47.62 GB - 
INFO:bittensor: - 45.0% of the GPU memory will be utilized for loading the model to device "CUDA". - 


2024-07-31 11:32:36.355 |       INFO       |  - Available free memory: 44.65 GB - 
2024-07-31 11:32:36.359 |       INFO       |  - Total gpu memory 47.62 GB - 
2024-07-31 11:32:36.362 |       INFO       |  - 45.0% of the GPU memory will be utilized for loading the model to device "CUDA". - 
2024-07-31 11:32:59.729 |       INFO       |  - 🤖 Generating query... - 
2024-07-31 11:33:00.124 |       INFO       |  - vLLM_LLM generated the following output:


Can you tell me more about Josh Banks? Who is he, and what's his story? What makes him interesting or noteworthy? - 
2024-07-31 11:33:00.127 |       INFO       |  - 🤖 Generating reference... - 
2024-07-31 11:33:00.403 |       INFO       |  - vLLM_LLM generated the following output:


The provided context is related to the topic of "Career". - 


WARNING 07-31 11:32:36 config.py:244] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 07-31 11:32:36 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='casperhansen/llama-3-8b-instruct-awq', speculative_config=None, tokenizer='casperhansen/llama-3-8b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=casperhansen/llama-3-8b-instruct-awq, use_v2_block_manager=False,

In [7]:
# as you can see, "pipeline" is an object that simply wraps around the LLM and is callable
pipeline("What's 1+2?")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.26s/it, est. speed input: 2.15 toks/s, output: 78.47 toks/s]


'"))  # -> "What\'s 1+2?"\nprint(eval("1+2"))  # -> 3\nprint(eval("True and False"))  # -> False\nprint(eval("True and (True or False)"))  # -> True\n```\n\nIn Python, `eval()` is a built-in function that parses the expression passed to this method and executes Python expression(s) passed as a string.\n\nHere are some important things to note about `eval()`:\n\n1.  `eval()` evaluates the expression in the current scope. If you want to evaluate an expression in a different scope, you need to use the `exec()` function instead.\n2.  `eval()` can execute arbitrary Python code. This makes it possible to execute any Python code as a string, but it also makes it a potential security risk, especially when dealing with untrusted input.\n3.  `eval()` is generally considered unsafe for use with untrusted input, such as user-supplied data, because it can execute any Python code. In particular, if the input contains a string that is used as an argument to `os.system()`, `subprocess.Popen()`, or `os

## Dataset

Datasets generate 'Context' objects, which contain a 'row' of data, in this case about wikipedia

In [1]:
from prompting.tools.datasets import WikiDataset
dataset = WikiDataset()
context = dataset.random()
context

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "name" in "WikiDateDataset" shadows an attribute in parent "BaseDataset"
  warnings.warn(


Context(title='Josh Banks', topic='Career', subtopic='Career', content='Career', internal_links=['Career', 'Personal life'], external_links=['2010 in baseball', 'Baseball', 'Major League Baseball', '2007 Major League Baseball season', '2010 Major League Baseball season', '2008 in baseball', 'Florida International University', 'San Diego Padres', 'Toronto Blue Jays', '2003 Major League Baseball Draft'], source='Wikipedia', tags=['1982 births', 'American expatriate baseball players in Canada', 'Auburn Doubledays players', 'Baseball players from Anne Arundel County, Maryland', 'Baseball players from Baltimore', 'Cotuit Kettleers players', 'Dunedin Blue Jays players', 'FIU Panthers baseball players', 'Fresno Grizzlies players', 'Houston Astros players', 'Living people', 'Long Island Ducks players', 'Major League Baseball pitchers', 'New Hampshire Fisher Cats players', 'People from Severna Park, Maryland', 'Portland Beavers players', 'Round Rock Express players', 'San Diego Padres players',

## Tasks

Tasks are objects that can be used to generate the query & reference for a miner

### Initialise with past data

We can either initialise the task with past data (this doesn't require an LLM to run)

In [ ]:
SummarizationTask(query="Summarize the following text: 'The quick brown fox jumps over the lazy dog.'", reference="A fox jumps over a dog")

### Create new task

Or we create a new task - this requires using the LLM pipeline

In [5]:
task = SummarizationTask(context=context, static_query=False)
task.generate_query(pipeline=pipeline)
task.generate_reference(pipeline=pipeline)

INFO:bittensor: - 🤖 Generating query... - 
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.63it/s, est. speed input: 84.72 toks/s, output: 74.13 toks/s]
INFO:bittensor: - vLLM_LLM generated the following output:


Can you tell me more about Josh Banks? Who is he, and what's his story? What makes him interesting or noteworthy? - 
INFO:bittensor: - 🤖 Generating reference... - 
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.79it/s, est. speed input: 883.35 toks/s, output: 53.53 toks/s]
INFO:bittensor: - vLLM_LLM generated the following output:


The provided context is related to the topic of "Career". - 


'\n\nThe provided context is related to the topic of "Career".'

## Miner Responses

Now let's say we have a few miners giving us responses

In [10]:
from prompting.dendrite import DendriteResponseEvent, SynapseStreamResult, StreamPromptingSynapse

miner_response_1 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="4", roles=["user"], messages=["What's 1+2?"]))
miner_response_2 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="3", roles=["assistant"], messages=["What's 1+2?"]))


# the synapses from all miners get collected into the DenriteResponseEvent
dendrite_response = DendriteResponseEvent(stream_results=[miner_response_1, miner_response_2], uids=[1, 2], timeout=10)

# Scoring

We can now pass the query, reference and miner responses to our scoring function, which is then responsible for giving each miner a score which is later used to set weights:

In [11]:
reward = SummarizationRewardConfig()
reward.apply(query="What's 1+2?", reference="1+2 is equal to 3", response_event=dendrite_response)

array([0.03413734, 0.17849946], dtype=float32)

# Other tests/examples on different tasks

In [13]:
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig
qa = QuestionAnsweringTask(context=context, llm_pipeline=pipeline, reward_config=SummarizationRewardConfig())

In [ ]:
qa.generate_query(pipeline=pipeline)
qa.generate_reference(pipeline=pipeline)

In [7]:
qa_task = QuestionAnsweringTask(llm_pipeline=pipeline, context=context)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s, est. speed input: 567.92 toks/s, output: 360.75 toks/s]


In [10]:
print(qa_task.reference)



{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>presenter<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you


In [10]:
TaskRegistry.tasks

[TaskConfig(), TaskConfig()]